In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm  # 用于显示进度条

In [ ]:
# 设置数据集路径(改成你的实际路径)
DATASET_PATH = "../gesture recognition/leapGestRecog"  # 改成你的路径

# 设置参数
TARGET_SIZE = (96, 96)  # 目标图片大小
NUM_CLASSES = 8        # 类别数
TRAIN_RATIO = 0.8      # 训练集比例
CAL_RATIO = 0.1        # 校准集比例
TEST_RATIO = 0.1       # 测试集比例

# 类别名称映射(方便后续使用)
CLASS_NAMES = {
    '00': 'palm',
    '01': 'l',
    '02': 'fist',
    '03': 'thumb',    # 原来是'04'
    '04': 'index',    # 原来是'05'
    '05': 'ok',       # 原来是'06'
    '06': 'c',        # 原来是'08'
    '07': 'down'      # 原来是'09'
}

In [ ]:
def preprocess_image(image_path):
    """
    图片预处理函数
    1. 读取图片
    2. 转换为灰度图
    3. 调整大小
    4. 归一化
    """
    # 读取图片
    img = cv2.imread(image_path)
    # 转换为灰度图
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 调整大小
    resized = cv2.resize(gray, TARGET_SIZE)
    # 归一化
    normalized = resized.astype('float32') / 255.0
    
    return normalized

def augment_image(image):
    """
    数据增强函数
    1. 随机旋转
    2. 随机缩放
    3. 随机亮度调整
    """
    # 这里先返回原图,后面我们再添加增强
    return image

# 测试函数是否正常工作
test_img_path = os.path.join(DATASET_PATH, "00", "01_palm", "frame_00_01_0001.png")
if os.path.exists(test_img_path):
    test_result = preprocess_image(test_img_path)
    print("预处理函数测试成功!")
    print("处理后图片形状:", test_result.shape)

In [ ]:
def collect_data():
    """收集并处理所有图片数据"""
    images = []
    labels = []
    gesture_folders = ['01_palm', '02_l', '03_fist', '05_thumb', 
                      '06_index', '07_ok', '09_c', '10_down']
    
    gesture_to_label = {
        '01_palm': 0,
        '02_l': 1,
        '03_fist': 2,
        '05_thumb': 3,
        '06_index': 4,
        '07_ok': 5,
        '09_c': 6,
        '10_down': 7
    }
    
    # 遍历所有数字文件夹
    for class_dir in [f"{i:02d}" for i in range(10)]:
        class_path = os.path.join(DATASET_PATH, class_dir)
        
        # 遍历目标手势文件夹
        for gesture_folder in gesture_folders:
            gesture_path = os.path.join(class_path, gesture_folder)
            if not os.path.exists(gesture_path):
                continue
                
            print(f"处理文件夹 {class_dir}: {gesture_folder}")
            
            for img_name in tqdm(os.listdir(gesture_path)):
                if img_name.endswith('.png'):
                    img_path = os.path.join(gesture_path, img_name)
                    processed_img = preprocess_image(img_path)
                    images.append(processed_img)
                    labels.append(gesture_to_label[gesture_folder])
    
    return np.array(images), np.array(labels)

# 执行数据收集
print("开始收集和处理数据...")
images, labels = collect_data()  # 保存返回值
print("\n数据收集完成!")
print("总图片数量:", len(images))
print("总标签数量:", len(labels))

In [ ]:
def split_and_save_data(images, labels):
    """划分数据集并保存"""
    # 首先分出测试集
    X_temp, X_test, y_temp, y_test = train_test_split(
        images, labels, 
        test_size=TEST_RATIO, 
        random_state=42,
        stratify=labels
    )
    
    # 从剩余数据中分出校准集
    X_train, X_cal, y_train, y_cal = train_test_split(
        X_temp, y_temp,
        test_size=CAL_RATIO/(TRAIN_RATIO + CAL_RATIO),
        random_state=42,
        stratify=y_temp
    )
    
    return {
        'train': (X_train, y_train),
        'cal': (X_cal, y_cal),
        'test': (X_test, y_test)
    }

# 执行数据集划分
datasets = split_and_save_data(images, labels)
print("数据集划分完成!")

In [ ]:
def verify_data(name, X, y):
    """验证数据对应关系"""
    print(f"\n验证{name}数据集:")
    # 1. 检查长度是否相等
    assert len(X) == len(y), f"{name}数据集中X和y长度不一致!"
    print(f"数据集大小: {len(X)}")
    
    # 2. 检查每个类别的数量
    for i in range(NUM_CLASSES):
        count = np.sum(y == i)
        print(f"类别{i} ({CLASS_NAMES[f'{i:02d}']}): {count}张图片")
    
    return True

# 验证所有数据集
for name, (X, y) in datasets.items():
    verify_data(name, X, y)

In [ ]:
# 保存验证后的数据集
for name, (X, y) in datasets.items():
    with open(f'{name}.pkl', 'wb') as f:
        pickle.dump((X, y), f)
    print(f'\n{name}.pkl 保存成功!')

print("\n所有数据处理和保存完成!")

In [ ]:
# 加载数据示例
with open('train.pkl', 'rb') as f:
    X_train, y_train = pickle.load(f)
    
with open('cal.pkl', 'rb') as f:
    X_cal, y_cal = pickle.load(f)
    
with open('test.pkl', 'rb') as f:
    X_test, y_test = pickle.load(f)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)

In [ ]:
# 加载并验证数据格式
with open('train.pkl', 'rb') as f:
    X_train, y_train = pickle.load(f)

print("数据类型:", X_train.dtype)  # 应该是 float32
print("数据范围:", X_train.min(), "-", X_train.max())  # 应该是 0-1
print("数据形状:", X_train.shape)  # (15999, 96, 96)

# 如果需要添加通道维度，可以这样做：
X_train_with_channel = X_train[..., np.newaxis]
print("添加通道维度后的形状:", X_train_with_channel.shape)  # (15999, 96, 96, 1)

# 显示一张图片看看是否正确
import matplotlib.pyplot as plt
plt.imshow(X_train[1500], cmap='gray')
plt.axis('off')
plt.show()